In [68]:
import numpy as np
import cv2
import glob

from scipy import linalg
import math

To capture images from webcam

In [2]:
# cam = cv2.VideoCapture(0)
# cv2.namedWindow("test")

# while True:
#     ret, frame = cam.read()
#     cv2.imshow("test", frame)
    
#     if not ret:
#         break
#     k = cv2.waitKey(1)

#     if k%256 == 27:
#         # ESC pressed
#         print("Escape hit, closing...")
#         break
#     elif k%256 == 32:
#         # SPACE pressed
#         img_name = "opencv_frame_{}.png".format(img_counter)
#         cv2.imwrite(img_name, frame)
#         print("{} written!".format(img_name))
#         img_counter += 1

# cam.release()

# cv2.destroyAllWindows()

Taking the image from the opencv webstite, as the object-3d-coordinates are mentioned for those pics

In [2]:
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)

In [3]:
objp.shape

(42, 3)

In [4]:
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('*.jpg')

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (7,6),None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        
        # termination criteria
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        
        objpoints.append(objp)
        cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners)

        # Draw and display the corners
        cv2.drawChessboardCorners(img, (7,6), corners,ret)
        cv2.imshow('img',img)
        cv2.waitKey(5000)
    
cv2.destroyAllWindows()

In [43]:
print(corners[:1])

[[[146.39856 240.75754]]]


We now have the image and object co-ordinates

### Calibration:

homogeneous eqn

In [5]:
imgp = np.ones((3,42))

imgp1 = np.array(imgpoints)
imgp1 = imgp1.reshape((2,42))
imgp[:2,:] = imgp1
print(imgp.shape)

objp = objp.T
print(objp.shape)
obj = np.ones((4,42))
obj[:3,:] = objp

(3, 42)
(3, 42)


Pseudo-inversion for calculation of camera matrix

In [6]:
objinv = np.linalg.pinv(obj)

camera_matrix = np.matmul(imgp,objinv)
print(camera_matrix)

[[-6.18933569e+00  2.26181529e+00  0.00000000e+00  1.81677088e+02]
 [-7.00757036e+00  9.66380777e-01  0.00000000e+00  2.30376379e+02]
 [ 2.77555756e-17 -6.93889390e-17  0.00000000e+00  1.00000000e+00]]


Verifying if the obtained matrix is correct by checking the orthogonal properties of camera matrix colmns

In [7]:
h1 = camera_matrix[:,0]
h2 = camera_matrix[:,1]
h1.reshape((1,3))
h2.reshape((3,1))

array([[ 2.26181529e+00],
       [ 9.66380777e-01],
       [-6.93889390e-17]])

In [8]:
# verifying if everything is correct before inversion

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)

m1 = np.matmul(mtx,mtx.T)

from numpy.linalg import inv
m1inv = inv(m1)

ans1 = np.matmul(h1,m1inv)
ans2 = np.matmul(ans1,h2)

print(ans2)

-0.00010207027118241976


This should have been zero.......verified, I guess..

Now decomposing camera matrix to find camera calibration matrix

In [15]:
print(camera_matrix)

[[-6.18933569e+00  2.26181529e+00  0.00000000e+00  1.81677088e+02]
 [-7.00757036e+00  9.66380777e-01  0.00000000e+00  2.30376379e+02]
 [ 2.77555756e-17 -6.93889390e-17  0.00000000e+00  1.00000000e+00]]


In [46]:
print(camera_matrix.shape)
camera_rotn = camera_matrix[:,:3]
camera_trans = camera_matrix[:,3:]
print("rotation_matrix\n",camera_rotn)
print(camera_rotn.shape)
print("translation_matrix\n",camera_trans)
print(camera_trans.shape)

(3, 4)
rotation_matrix
 [[-6.18933569e+00  2.26181529e+00  0.00000000e+00]
 [-7.00757036e+00  9.66380777e-01  0.00000000e+00]
 [ 2.77555756e-17 -6.93889390e-17  0.00000000e+00]]
(3, 3)
translation_matrix
 [[181.67708768]
 [230.37637939]
 [  1.        ]]
(3, 1)


In [51]:
# checking if translation matrix is correct
p2 = np.matmul(mtx,tvecs)
p2 = p2/13
p2

array([[[152.22669317],
        [254.19586228],
        [  1.0509125 ]]])

The values are not matching

Checking for rotation matrix

In [28]:
# applying RQ decomposition

kp, r = linalg.rq(camera_rotn)
np.allclose(camera_rotn, kp @ r)
print("k-shape",kp.shape)
print("r-shape", r.shape)
print(kp)

k-shape (3, 3)
r-shape (3, 3)
[[ 6.22344726e-16  4.90663681e+00  4.39870434e+00]
 [ 0.00000000e+00  6.14746093e+00  3.49980832e+00]
 [ 0.00000000e+00  0.00000000e+00 -7.47341745e-17]]


kp is not in correct format. (0,1) and (2,2) should have been 0 and 1 respectively

In [52]:
# print(kp[2,2])

# factor = 1/kp[2,2]
# # factor = (-1)*factor
# factor

# # this term needs to be one
# k = kp*factor
# k

In [63]:
rvecs[0][0]

array([-0.44702636])

In [69]:
rx = [[1,0,0],[0,math.cos(rvecs[0][0]),math.sin(rvecs[0][0])],[0,(-1)*math.sin(rvecs[0][0]),math.cos(rvecs[0][0])]]
ry = [[math.cos(rvecs[0][0]), 0, math.sin(rvecs[0][0])],[0,1,0],[(-1)*math.sin(rvecs[0][0]), 0, math.cos(rvecs[0][0])]]
rz = [[math.cos(rvecs[0][0]),math.sin(rvecs[0][0]),0],[(-1)*math.sin(rvecs[0][0]),math.cos(rvecs[0][0]),0],[0,0,1]]

In [70]:
rx

[[1, 0, 0],
 [0, 0.9017365492475191, -0.43228601151457186],
 [0, 0.43228601151457186, 0.9017365492475191]]

In [72]:
rotn1 = np.matmul(rx,ry)
rotn2 = np.matmul(rotn1,rz)
rotn2

array([[ 0.8131288 , -0.3898081 , -0.43228601],
       [ 0.22129951,  0.89391061, -0.3898081 ],
       [ 0.5383754 ,  0.22129951,  0.8131288 ]])

rotn2 = p2

### Calibration: OpenCV

In [20]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)

In [13]:
mtx

array([[442.97606554,   0.        , 225.9079943 ],
       [  0.        , 469.44361439, 163.99574565],
       [  0.        ,   0.        ,   1.        ]])

This is the camera matrix

In [15]:
print(ret)
print(dist)
print(rvecs)
print(tvecs)

0.10283405403602608
[[-4.18191883e-01  1.22064773e+01 -5.62393390e-04  1.37321681e-02
  -1.26093230e+02]]
[array([[-0.44702636],
       [-0.32636351],
       [-1.54572073]])]
[array([[-2.49985727],
       [ 2.26663832],
       [13.66186248]])]


**Undistortion**

## **Undistortion**

In [23]:
img = cv2.imread('chess1.jpg')
h, w = img.shape[:2]
newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))

We have got what we were trying. Now we can take an image and undistort it.

In [24]:
# undistort
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

# crop the image
x,y,w,h = roi
dst = dst[y:y+h, x:x+w]
cv2.imwrite('calibresult.png',dst)

False

USE REMAPPING

In [25]:
# undistort
mapx,mapy = cv2.initUndistortRectifyMap(mtx,dist,None,newcameramtx,(w,h),5)
dst = cv2.remap(img,mapx,mapy,cv2.INTER_LINEAR)
 
# crop the image
x,y,w,h = roi
dst = dst[y:y+h, x:x+w]
cv2.imwrite('calibresult.png',dst)

error: C:\ci\opencv_1512688052760\work\modules\imgproc\src\imgwarp.cpp:1728: error: (-215) _map1.size().area() > 0 in function cv::remap


REPROJECTION ERROR

In [ ]:
mean_error = 0
for i in xrange(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv2.norm(imgpoints[i],imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    tot_error += error

print "total error: ", mean_error/len(objpoints)